<a href="https://colab.research.google.com/github/tuonglab/scRNAseq-tutorial/blob/master/notebook/tuong_dpt_TCR_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# scRNA-seq analysis in-depth tutorial for immunology/T cell data!

This analysis notebook is meant to be a supplement to the existing tutorials out there but with a more in-depth focus on how we normally analyse immunology data. The example we will go through below is that of sorted T cell populations from human PBMC and we will go through the details as we progress. For now, we will do the first step which is to set-up the prerequisites and download the data:

In [ ]:
# setup the notebook
!pip install -qqq scanpy[leiden] sc-dandelion bbknn
!pip install -qqq git+https://www.github.com/tuonglab/kttools.git
!pip install -qqq celltypist
# the session might crash after running this step as we are installing numpy. Don't worry. just reconnect and then continue with the rest below.
!celltypist --update-models
# second thing to do is to clone the repository so that we have all the data and notebooks ready to go
!git clone https://github.com/tuonglab/scRNAseq-tutorial.git

In [ ]:
# change directory
import os

os.chdir("scRNAseq-tutorial")
[file for file in os.listdir() if not file.startswith(".")]

In [ ]:
# just to make the the code run a bit quieter, i'm suppressing a few warnings here

import warnings

from numba.core.errors import NumbaDeprecationWarning, NumbaWarning

for warn in [
    NumbaDeprecationWarning,
    NumbaWarning,
    FutureWarning,
    UserWarning,
]:  # otherwise very noisy
    warnings.filterwarnings("ignore", category=warn)

# Tutorial: Single-cell RNA seq analysis using a T cell dataset (sorted from healthy PBMCs)

#### This tutorial will show you the steps involved to get you started on single cell analysis in Python using Scanpy, the toolkit for analysing single-cell gene expression.
#### It includes preprocessing, visualization, clustering, trajectory inference and differential expression testing.
#### Depending on the complexity of your analysis, you may need to modify and expand on these steps.

# Double-Positive T scRNAseq analysis

## 1. Preprocessing and Quality Control

First, import packages needed for single-cell RNA seq analysis.

You can insert another line: `pip install "package_name"` or install them via conda/mamba/anaconda if those packages are not already installed prior to importing

In [ ]:
import os
import tools
import scanpy as sc
import pandas as pd
import dandelion as ddl
from tqdm import tqdm

Show current directory:

In [ ]:
os.getcwd()

Print software versions:

In [ ]:
sc.logging.print_header()

In [ ]:
ddl.logging.print_header()

Set plotting parameters:

In [ ]:
sc.settings.set_figure_params(dpi=80, facecolor="white")

### Reading in files for analysis

For this tutorial, we have already saved the starting raw datafile as an `.h5ad` file. Hence we are reading it in with `sc.read_h5ad` to load up an `AnnData` object. However, for a normal use case, you would typically start with `cellranger` output files/folders. Therefore, refer to their [tutorial](https://scanpy.readthedocs.io/) to get an idea on how to read other file formats in.

In [ ]:
# we need to read each file separately and then concatenate them

from pathlib import Path

samples = [
    "CD4_L01",
    "CD4_P01",
    "CD4_WY",
    "CD8_L01",
    "CD8_P01",
    "CD8_WY",
    "DP_L01",
    "DP_P01",
    "DP_WY",
    "V_CD4",
    "V_CD8",
    "V_DP",
]

# use a for loop to read in each sample and then concatenate at the end
adatas = []
for sample in samples:
    out_path = Path("data") / (sample + "_raw.h5ad")  # "../" means go up one folder
    adata = sc.read_h5ad(out_path)
    adatas.append(adata)

### Concatenate

Now that we have read them in and added them to a list, we want to join them all up (also known as concatenate). But how do we do that?

In [ ]:
adata = sc.concat(adatas)
adata

Note: this adata has gone through some pre-processing CALCULATIONS but have NOT been filtered...

## Quality control


These are the steps that Kelvin did to obtain some preprocessing parameters AFTER concatenating raw output from cell ranger, BEFORE saving it as "raw_adata_v2.h5ad"

When Kelvin ran
`ddl.pp.external.recipe_scanpy_qc(
        tmp, max_genes=10000, mito_cutoff=None, mito_startswith="MT-"
)`, he applied a quick "recipe" function from ddl to help compute PP output using these parameters in this context: {cells need to have 10,000 genes, and no mito_cutoff done}. It uses a gaussian mixture model on the mitochondrial and total gene counts to decide which cells pass or not.

It then creates a "filter_rna" column to store the final outoput, whether a cell passes QC (False), or not (True).




❓#### What is a way to quickly look at the cells in our metadata with the observational file?

Hint - where does `AnnData` store the `metadata`? What `type` is it?

In [ ]:
adata.obs

#### What about genes in the metadata?

Hint - in the variable level

In [ ]:
adata.var

Now let's visualise some QC metrics. Ask yourself what is each plot trying to tell you? Eg Plot 1 = cells with highest mito content tend to also have lower read counts, suggesting low quality cells.

In [ ]:
sc.pl.scatter(adata, x="total_counts", y="pct_counts_mt", color="filter_rna")
sc.pl.scatter(adata, x="total_counts", y="n_genes_by_counts", color="filter_rna")
sc.pl.scatter(adata, x="pct_counts_mt", y="n_genes_by_counts", color="filter_rna")

 - `n_genes_by_counts` = number of genes with positive counts in a cell (similar to `nFeature_RNA` in seurat)

 - `total_counts` = total number of counts for a cell, which is also known as library size (similar to `nCount_RNA` in seurat = nUMI)

🔔 Look out for bell signs as they are extra handy tips for coding in general!
**Tabulate number of cells per sample...**

In [ ]:
pd.crosstab(adata.obs["sample"], adata.obs["filter_rna"])

# cross tab: https://stackoverflow.com/questions/21247203/how-to-make-a-pandas-crosstab-with-percentages
# https://pandas.pydata.org/docs/reference/api/pandas.crosstab.html
# https://pbpython.com/pandas-crosstab.html

In [ ]:
## Repeating this as a percentage
# You'll just have the function that does row/row.sum(), and you use apply with axis=1 to apply it by row (or across and not within columns).

pd.crosstab(adata.obs["sample"], adata.obs["filter_rna"]).apply(
    lambda row: row / row.sum() * 100, axis=1
)

In [ ]:
## Plotting violin plots for QC metrics too

sc.pl.violin(
    adata,
    ["total_counts", "n_genes_by_counts", "pct_counts_mt"],
    size=0.1,
    groupby="sample",
    multi_panel=True,
    rotation=90,
)

Continue processing with "good" cells only..

In [ ]:
adata = adata[adata.obs["filter_rna"] == "False"].copy()

In [ ]:
## always check after you have done some subsetting to make sure changes are applied correctly..
adata  # i filtered out ~10K cells

❓> Ask yourself, what are usual good cut-offs? Are they always appropriate?

Redo QC plots with the "good" cells post-filtering

In [ ]:
sc.pl.scatter(adata, x="total_counts", y="pct_counts_mt", color="filter_rna")
sc.pl.scatter(adata, x="total_counts", y="n_genes_by_counts", color="filter_rna")
sc.pl.scatter(adata, x="pct_counts_mt", y="n_genes_by_counts", color="filter_rna")

In [ ]:
# also via violin plot

sc.pl.violin(
    adata,
    ["total_counts", "n_genes_by_counts", "pct_counts_mt"],
    size=0.1,
    groupby="sample",
    multi_panel=True,
    rotation=90,
)

🔔*The steps below are optional - So this is me thinking about how generic workflows would apply some cutoff on things like mito content. I want to see if it helps the dataset better as I can see in my violin plots there are some cells with >5% mito content.*




In [ ]:
# first i make an extra copy of the adata so I don't overwrite the MAIN object I am working on.
bdata = adata.copy()
bdata

In [ ]:
bdata = bdata[bdata.obs.pct_counts_mt < 5, :]
# I also want to filter out cells with low gene expression..
bdata = bdata[bdata.obs.n_genes_by_counts > 200, :]
bdata  # here I can see that it has only filtered out another 20+ cells suggesting minimal changes, and the original QC metrics applied was probably stringent enough.

In [ ]:
# also violin plot

sc.pl.violin(
    bdata,
    ["total_counts", "n_genes_by_counts", "pct_counts_mt"],
    size=0.1,
    groupby="sample",
    multi_panel=True,
    rotation=90,
)

# didn't change too much - so ignore..

### **I am happy with the original QC metrics used. And will go back to my original data stored as `adata`**

In [ ]:
# Remove uninformative genes - this filters out genes that are not expressed in least 3 cells.

sc.pp.filter_genes(adata, min_cells=3)
adata

`n_obs` basically means number of observations (cells) and `n_vars` is number of variables (genes).

## Normalisation

🔔 #### OPTIONAL - THESE NEXT FEW STEPS IS JUST TO ILLUSTRATE WHAT LOG-NORMALISATION DOES AND WHAT YOU SHOULD EXPECT..

In [ ]:
# create a separate dataframe to compare.. this df is the gex matrix BEFORE normalisation
prenorm_df = adata.to_df()
prenorm_df

In [ ]:
## OPTIONAL - Now, prior to normalisation, gene values can range from 0 - 10K/100K reads.
# To see this, we can either perform a rowsum of all genes for each cell or just ask what is the max read of a gene for each cell.
# Calculate row sums and find the maximum
max_row_sum = prenorm_df.sum(axis=1).max()
print("Max total gene counts in a cell is", max_row_sum)

# Find the maximum value in all rows combined
max_value_all_rows = prenorm_df.max().max()
print("Max gene count in a cell is", max_value_all_rows)

### Perform Normalisation:
Total-count normalize (library-size correct) the data matrix 𝐗 to 10,000 reads per cell, so that counts become comparable among cells.

In [ ]:
sc.pp.normalize_total(adata, target_sum=1e4)

# Logarithmize the data:
sc.pp.log1p(adata)

# check the object
adata

❓> What are some changes that you can see to confirm that normalisation has taken
place? HINT - "look for log1p" Are there any other ways to confirm?

In [ ]:
## OPTIONAL - We can look into our gene matrix again to manually confirm....

postnorm_df = adata.to_df()
postnorm_df

In [ ]:
## OPTIONAL - Now, post-normalisation, gene values are a lot "narrower" in range.
# To see this again, we can either perform a rowsum of all genes for each cell or just ask what is the max read of a gene for each cell.
# Calculate row sums and find the maximum
max_row_sum = postnorm_df.sum(axis=1).max()
print("Max total gene counts in a cell is", max_row_sum)

# Find the maximum value in all rows combined
max_value_all_rows = postnorm_df.max().max()
print("Max gene count in a cell is", max_value_all_rows)

# 2. Highly variable Genes

Identify and inspect highly-variable genes

In [ ]:
# (Expects logarithimised data)
sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5)
adata

In [ ]:
# to see genes
adata.var  # there is now a hvg column

##### 🔔to get structure of dataframe

In [ ]:
adata.var.info()
adata.obs.info()

##### 🔔 to get the number of HV genes... because the column "highly_variable" is a boolean vector - ie it gives True (1) or False (0), you can use sum function to know...

In [ ]:
print(adata.var.highly_variable.sum())

Plot out the number of highly variable genes

In [ ]:
sc.pl.highly_variable_genes(adata)

🔔 Other ways to inspect HV genes matrix, first, create a dataframe containing the info.

In [ ]:
hvg = pd.DataFrame(adata.var.highly_variable)

🔔 Add col name 'HV_status' and print hvg of only True values.

In [ ]:
hvg.columns = ["HV_status"]
hvg_T = hvg.loc[hvg["HV_status"] == True]
hvg_T

🔔 how to list all variables you have so far

In [ ]:
%who_ls

## Removing VDJ genes from HVG genes manually

### So how does T-cell receptor (TCR) differ from B-cell receptor (TCR)?
TCR has a single pair of alpha and beta chains while BCR contains two pairs of light and heavy chains.

In the examples below, TCR can be expressed as TR while BCR can be expressed as IG.

**The code below is for BCR data.**

Because this is 10x 5' data - there is a lot of VDJ genes that are picked up (in abundance), and because they do look different from each donor, it can be mis-computed as HVG - therefore, <b> REMOVE THEM </b>.

In [ ]:
import re

for i in adata.var.index:
    if re.search("^IG[HKL][VDJC]", i):
        adata.var.at[i, "highly_variable"] = False

It uses the `re` module, which stands for regular expressions, to search for specific patterns in the index values.

Let's break down the code step by step:

 - `for i in adata.var.index` -> This initiates a loop that iterates over each value (i) in the index of the adata.var object.

 - `if re.search("^IG[HKL][VDJC]", i)`
    - This line uses regular expressions to check if the current index value (i) matches a specific pattern. The regular expression pattern is as follows:

      - `^`: Asserts the position at the start of the string.
      - `IG`: The first two characters match the characters "IG" literally.
      - `[HKL]`: The next character matches either "H", "K", or "L".
      - `[VDJC]`: The next character either "V", "D", "J", or "C".      

Therefore, the regular expression is looking for index values that either start with "IG", followed by "H", "K", or "L", and then one of "V", "D", "J", or "C".

  - `adata.var.at[i, "highly_variable"] = False` : If the regular expression matches the current index value (i), it sets the corresponding value of "highly_variable" in the adata.var object to False.

In summary, this modified code is looping through the index values of the adata.var object and using a regular expression to identify specific patterns in the index values that match either the BCR V/D/J gene pattern. If a match is found, it sets the corresponding "highly_variable" value to False. This code could be used to mark certain entries in the adata.var object as not highly variable based on the new regular expression pattern. Just like before, it's important to ensure that you understand the data and the intention behind the code before executing it, as regular expressions can be tricky and their behavior can be sensitive to small changes in the pattern.

The purpose of this code seems to be to mark certain entries in the adata.var object as "not highly_variable" based on the matching pattern defined in the regular expression.

### Now, repeat the code but only do it from the TCR dataset

Do you understand what the regex pattern is looking for now?

In [ ]:
import re

for i in adata.var.index:
    if re.search("^TR[AB][VDJ]", i):
        adata.var.at[i, "highly_variable"] = False

### Now how to use one regex pattern to catch both TCR and BCR?

In [ ]:
import re

for i in adata.var.index:
    if re.search("^TR[AB][VDJ]|^IG[HKL][VDJC]", i):
        adata.var.at[i, "highly_variable"] = False

Let's re-inspect the highly-variable gene list

In [ ]:
hvg = pd.DataFrame(adata.var.highly_variable)
hvg_T = hvg.loc[hvg["highly_variable"] == True]
hvg_T  # 2915 genes...

In [ ]:
sc.pl.highly_variable_genes(adata)

### Explain what adata.raw, adata.layers and adata.X mean; + include examples after 40 using numpy - normal vs log-transformed
✅ Megan: i have done up examples of normal vs log-transformed in the "NORMALISATION" SECTION. Not with numpy though...

### What is the difference between adata.raw, adata.X, and adata.raw.X?


`adata.raw` = Setting the `.raw` attribute of the `AnnData` object to the normalized and logarithmized raw gene expression for later use in differential testing and visualizations of gene expression. This simply freezes the state of the AnnData object.

The raw attribute is initialized with the current content of an object by setting:

In [ ]:
adata.raw = adata

You can get back an AnnData of the object in .raw by calling .raw.to_adata().

In [ ]:
test_data = adata.raw.to_adata()
test_data

adata.X represents the data matrix of n_obs × n_var and so is referencing an explicit layer of .X

But in our case we will need the .raw attribute to run the next line

# 3. Perform Dimensionality Reduction

## 3a. PCA

In [ ]:
# Actually do the filtering for PCA
adata = adata[
    :, adata.var.highly_variable
].copy()  # all rows/indexes : columns only selecting for those within high_variable, and then making a copy of itself
adata

#### Regress out effects of "total_counts" per cell and percentage of mt genes expressed ("pct_counts_mt").

In [ ]:
sc.pp.regress_out(adata, ["total_counts", "pct_counts_mt"])

#### Scale each gene to unit variance. Clip values exceeding std of 10.

In [ ]:
sc.pp.scale(adata, max_value=10)

>❓Why do we need to scale the data?

🔔 OPTIONAL - Scaling the data adjusts the expression values of genes across cells to make them more comparable. How do we confirm?

In [ ]:
## OPTIONAL
# It often involves a centering and scaling step, which will typically give negative - positive values in a small range.

scaled_df = adata.to_df()
print(scaled_df[5:])

# To see this, we will look at the min, max and max sum of gene values for each cell.

# Get the min values in all rows combined
min_value_all_rows = scaled_df.min().min()
print("Min gene count in a cell is", min_value_all_rows)

# Calculate row sums and find the maximum
max_row_sum = scaled_df.sum(axis=1).max()
print("Max total gene counts in a cell is", max_row_sum)

# Find the maximum value in all rows combined
max_value_all_rows = scaled_df.max().max()
print("Max gene count in a cell is", max_value_all_rows)

In [ ]:
## get PCA
sc.tl.pca(adata, svd_solver="arpack")

In [ ]:
# get elbow plot to see variance contribution by each pc
sc.pl.pca_variance_ratio(adata, log=True, n_pcs=50)

In [ ]:
## Computing neighbourhood graph - here UMAP
sc.pp.neighbors(adata, n_neighbors=10, n_pcs=40)

## n_neighbor: The size of local neighborhood (in terms of number of neighboring data
# points) used for manifold approximation. Larger values result in more
# global views of the manifold, while smaller values result in more local
# data being preserved. In general values should be in the range 2 to 100.
# If `knn` is `True`, number of nearest neighbors to be searched. If `knn`
# is `False`, a Gaussian kernel width is set to the distance of the
# `n_neighbors` neighbor.

## 3b. UMAP Visualisation

Embedding the neighbourhood graph using UMAP

In [ ]:
sc.tl.umap(adata, min_dist=0.3)  # 0.3 is personal preference

#### Visualise UMAP:

In [ ]:
sc.pl.umap(
    adata,
    color=[
        "sample",
        "sort_type",
        "donor",
    ],
    ncols=1,
)

🔔 plotting multiple small umaps of the same parameter, similar to faceting/split.by
##### https://github.com/scverse/scanpy/issues/955

In [ ]:
def cluster_small_multiples(
    adata, clust_key, size=30, frameon=False, legend_loc=None, **kwargs
):
    tmp = adata.copy()

    for i, clust in enumerate(adata.obs[clust_key].cat.categories):
        tmp.obs[clust] = adata.obs[clust_key].isin([clust]).astype("category")
        tmp.uns[clust + "_colors"] = ["#d3d3d3", adata.uns[clust_key + "_colors"][i]]

    sc.pl.umap(
        tmp,
        groups=tmp.obs[clust].cat.categories[1:].values,
        color=adata.obs[clust_key].cat.categories.tolist(),
        size=size,
        frameon=frameon,
        legend_loc=legend_loc,
        **kwargs
    )

In [ ]:
cluster_small_multiples(adata, "sort_type")

In [ ]:
cluster_small_multiples(adata, "donor")

Next, I want to visualise key lineage genes on the UMAP that is subset by their sort_type.

####❓ Can I do a quick QC: subset data by sort type, and plot the relevant marker genes on them?

We can create an object by sorting for CD4 and then check the table of variables:

In [ ]:
adata.obs.info()
adata_CD4 = adata[adata.obs["sort_type"] == "CD4"]
adata_CD4.obs

## What are relevant genes for CD4 and CD8?

### Plot CD8 on adata_CD4
Check what CD8 gene is annotated as...

How would you colour the plot by?

(Instruction to host: Remove everything after colour =...)

In [ ]:
sc.pl.umap(adata_CD4, color=["sort_type", "CD4", "CD8B"], ncols=3)

### How would you sort for CD8?
#### Create an object for CD8 sorted cells and plot it as a UMAP


In [ ]:
adata_CD8 = adata[adata.obs["sort_type"] == "CD8"]
adata_CD8.obs
sc.pl.umap(adata_CD8, color=["sort_type", "CD4", "CD8B"], ncols=3)

### Repeat for DP sort..

In [ ]:
adata_DP = adata[adata.obs["sort_type"] == "DP"]
adata_DP.obs
sc.pl.umap(adata_DP, color=["sort_type", "CD4", "CD8B"], ncols=3)

## expression of CD4 in general isn't great... But most of the DP sorted cells seem to sit closer to the middle where CD4-sorted cells are.

## Another way of plotting...

## How would you visualise different T cell marker genes? ❓

Earlier, we saw genes CD4, CD8B. What are other cell surface markers found on T cells?

Some hints from https://www.nature.com/articles/s41467-019-12464-3

In [ ]:
sc.pl.umap(
    adata,
    color=[
        "CD4",
        "CD8B",
        "FOXP3",
        "SELL",
        "CCR7",
        "MKI67",
        "NKG7",
        "GATA3",
        "RORC",
        "CXCR5",
        "CD69",
        "GZMK",
    ],
    ncols=3,
    size=5,
    color_map=tools.cmp(),
)

>>❓ **POP QUIZ: What does it mean when you try plotting a gene and it spits out an error to say that the gene cannot be found? **

**🔔** Note: This is something for later in "BATCH CORRECTION"

In [ ]:
adata.uns["log1p"]
# {'base': None}

Let's just save this...

In [ ]:
adata.write_h5ad("adata_processed_v2_MS_070823.h5ad", compression="gzip")

# 4. Batch Correction

##### There are a number of methods or algorithms that can combine diverse datasets and remove batch effects. We call that batch correction.

##### Batch balanced k nearest neighbour (BBKNN) is a fairly recent algorithm that is "one to two orders of magnitude faster than existing methods". More info: https://academic.oup.com/bioinformatics/article/36/3/964/5545955

##### BBKNN is batch effect removal tool that can be directly used in the scanpy workflow.
##### It serves as an alternative to scanpy.pp.neighbors(), with both functions creating a neighbour graph for subsequent use in clustering, pseudotime and UMAP visualisation

>❓Why do we want to batch-correct? Especially in cases where there are "variation" in your dataset such as multiple donors, you will usually see these coming up and influencing your data structure.

In [ ]:
cluster_small_multiples(adata, "donor")

## Batch correction with BBKNN

Running BBKNN
https://scanpy-tutorials.readthedocs.io/en/latest/integrating-data-using-ingest.html

To run BBKNN, you can either continue on from before, or re-start by importing the object you have previously saved. NOTE: No batch-correction has been done prior to this.

In [ ]:
adata = sc.read_h5ad("adata_processed_v2_MS_070823.h5ad")

In [ ]:
adata.uns["log1p"]

🔔 Note it is empty now - this happens after the save h5ad step! https://github.com/scverse/scanpy/issues/2239    And if you continue from here, there is an issue as BBKNN doesn't like this slot to be empty..

Install `bbknn` if you haven't. Just uncomment the next line and run it.

In [ ]:
# !pip install bbknn

#### Apply bbknn to batch correct by sample

In [ ]:
sc.external.pp.bbknn(adata, batch_key="sample")

🔔 If you didn't read in the saved h5ad `adata_processed_v2_MS_070823.h5ad`, but just continued from the workflow PRIOR to "Batch-correction" then your output of `adata.uns.pop("log1p")= {'base' : None}` which won't be an issue.
But if not, this code chunk will automatically remove the problematic slot if it's there.

In [ ]:
try:
    adata.uns.pop("log1p")
except:
    pass

### Plotting the neighbourhood graph using bbknn

Note: we skip neighbourhood construction step as bbknn does that already
min_dist : float (default: 0.5).

The effective minimum distance between embedded points. Smaller values will result in a more clustered/clumped embedding where nearby points on the manifold are drawn closer together, while larger values will result on a more even dispersal of points. The value should be set relative to the spread value, which determines the scale at which embedded points will be spread out.

The default in the umap-learn package is 0.1. Try inserting different values to find your personal preference in the next few steps through visualisation.

In [ ]:
sc.tl.umap(adata, min_dist=0.3)

#### ❓How would you visualise this across the three main T cell types? You can do a loop too

In [ ]:
for g in ["CD4", "CD8", "DP"]:
    sc.pl.umap(
        adata,
        color=["sort_type"],
        groups=g,
        na_in_legend=False,
    )

#### How would you visualise by donor? Create a different loop this time and plot the UMAP:

In [ ]:
adata.obs["donor"]  # L01, P01, V, WY
# visualise this..
for d in ["L01", "P01", "V", "WY"]:
    sc.pl.umap(
        adata,
        color=["donor"],
        groups=d,
        na_in_legend=False,
    )

### Inspect the changes post-batch correction.

#### Now how do you sort for CD4 T cells? And then display them in UMAPs?

In [ ]:
adata_CD4 = adata[adata.obs["sort_type"] == "CD4"]
adata_CD4.obs
sc.pl.umap(adata_CD4, color=["sort_type", "donor", "CD4", "CD8B"], ncols=4, size=20)

### Inspect changes post-batch correction

#### Now how do you sort for CD8 T cells? And then display them in UMAPs?

In [ ]:
adata_CD8 = adata[adata.obs["sort_type"] == "CD8"]
adata_CD8.obs
sc.pl.umap(adata_CD8, color=["sort_type", "donor", "CD4", "CD8B"], ncols=4, size=20)

#### Now how do you sort for DP T cells? And then display them in UMAPs?

In [ ]:
adata_DP = adata[adata.obs["sort_type"] == "DP"]
adata_DP.obs
sc.pl.umap(adata_DP, color=["sort_type", "donor", "CD4", "CD8B"], ncols=4, size=20)

❓Why are there CD4-sorted T cells also found in the CD8 cluster, but they are with low expression of CD4??

As a starting point, let's visualise some QC metrics.. if needed, we can just recluster and compare downstream. The tail bit of umap seems to be a bit higher in mito content, but otherwise nothing seems too out of the blue.


In [ ]:
adata.obs
# n_genes_by_counts, total_cpunts, pct_count_mt, filter_rna

sc.pl.umap(
    adata,
    color=["n_genes_by_counts", "total_counts", "pct_counts_mt"],
    ncols=3,
    size=20,
    color_map=tools.cmp(),
)

### How do you visualize the expression of common T cell markers across the entire data set in three columns?

🔔 Hint: Using the same markers as we did earlier?

In [ ]:
sc.pl.umap(
    adata,
    color=[
        "CD4",
        "CD8B",
        "FOXP3",
        "SELL",
        "CCR7",
        "MKI67",
        "NKG7",
        "GATA3",
        "RORC",
        "CXCR5",
        "CD69",
        "GZMK",
    ],
    ncols=3,
    size=5,
    color_map=tools.cmp(),
)

### Comments:✋
These cells are not too different transcriptionally - because they are simply T cells?
Also, umap seems to cluster more based on "activation status" and then "cell lineage"
But we can see that most of the DP cells are located within the middle where the bulk of CD4 cells are.

## Save post-BBKNN

In [ ]:
adata.write_h5ad("adata_processed_v2_BBKNN_MS_100823.h5ad", compression="gzip")

In [ ]:
sc.external.pp.bbknn(adata, batch_key="donor")

In [ ]:
sc.tl.umap(adata, min_dist=0.3)

### How do you visualise CD4, CD8 and DP T cells by donor post-BBKNN batch correction?

In [ ]:
for g in ["CD4", "CD8", "DP"]:
    sc.pl.umap(
        adata,
        color=["sort_type"],
        groups=g,
        na_in_legend=False,
    )

### How to visualise individual UMAPS based of each donor type?

In [ ]:
adata.obs["donor"]  # L01, P01, V, WY

for d in ["L01", "P01", "V", "WY"]:
    sc.pl.umap(
        adata,
        color=["donor"],
        groups=d,
        na_in_legend=False,
    )

# Read in Data post-BBKNN

In [ ]:
adata = sc.read_h5ad("adata_processed_v2_BBKNN_MS_100823.h5ad")
try:
    adata.uns.pop("log1p")
except:
    pass

In [ ]:
sc.tl.umap(adata, min_dist=0.3)

#### Visualise this, you can do it with a loop too:

In [ ]:
for g in ["CD4", "CD8", "DP"]:
    sc.pl.umap(adata, color=["sort_type"], groups=g, size=20)

### How do I view proportion of CD4 expression in CD8+ cells? ❓

🔔 Hint: you can use a method of score_genes under scanpy. Then visualise the score by plotting the umap.

In [ ]:
sc.tl.score_genes(adata, gene_list=["CD4", "CD8B", "CD8A"], score_name="CD4.CD8.score")

In [ ]:
sc.pl.umap(adata, color="CD4.CD8.score", size=10)

### Now that we've seen what a batch corrected data is visualised on a UMAP, let's compare it with the UMAP of our adata:
We'll name the previous non-batch corrected data as "preadata"

In [ ]:
preadata = sc.read_h5ad("adata_processed_v2_MS_070823.h5ad")
try:
    preadata.uns.pop("log1p")
except:
    pass

In [ ]:
sc.tl.umap(preadata, min_dist=0.3)

In [ ]:
sc.tl.score_genes(
    preadata, gene_list=["CD4", "CD8B", "CD8A"], score_name="CD4.CD8.score"
)

In [ ]:
sc.pl.umap(preadata, color="CD4.CD8.score", size=10)

❓ - So can you see the difference in the UMAPs pre- and post-batch correction?

Now, moving back to our batch corrected adata object, let's check the annotations of variables:

In [ ]:
adata.var

> 🔔 Next, let's use an existing CD4/CD8 gene set from Szabo et al. to score the expression of various CD4/CD8 markers on our dataset.

> Read in the csv file and label it "szabo"

In [ ]:
szabo = pd.read_csv("geneset/Szabo_T_cell_gene_modules.txt", sep="\t")
szabo

Embed a score to the Szabo's "CD4/CD8" gene list and name the score as "CD4/CD8_geneset"

In [ ]:
sc.tl.score_genes(adata, gene_list=szabo["CD4/CD8"], score_name="CD4/CD8_geneset")

# So if it scores higher = it is more CD4-like...

Plot the usual CD4, CD8 genes together with two other T cell markers: CD44 and CD27 which are involved in acquiring activation, whereas CD27 is high on naive cells and lost upon activation.

In [ ]:
sc.pl.umap(
    adata,
    color=["CD4/CD8_geneset", "sort_type", "CD4", "CD8B", "CD27", "CD44"],
    ncols=3,
    size=5,
)

✋ It's a bit weird but the CD4/CD8 marker expression doesn't fully coincide with the CD4/CD8 lineage of sorted identities.

Let's do unsupervised clustering and see if there is any way we can understand CD4 vs CD8 on clusters.

# 5. Redo UMAP

# Try running Leiden unsupervised clustering:

Run leiden clustering with scanpy (BBKNN already has a neighbourhood construction step so can skip this)

In [ ]:
sc.tl.leiden(adata, resolution=2.0)

#### Plot a UMAP and choose a resolution ("size") of 20 clusters:

Reference for what other types of resolution mean:

resolution 1.0 (10 clusters), resolution 1.5 (12 clusters), resolution 1.8 (15 clusters), resolution 2.0 (16 clusters) - but looks heaps better!


In [ ]:
sc.pl.umap(adata, color="leiden", size=20)

In [ ]:
# putting legend on top
sc.pl.umap(adata, color="leiden", legend_loc="on data", legend_fontoutline=2, size=10)

In [ ]:
### just getting how many cells are there per cluster
adata.obs.groupby(["leiden"]).apply(len)

Creating bar plot distribution per cluster

In [ ]:
import matplotlib.pyplot as plt

#  https://pbpython.com/pandas-crosstab.html
# basically, cross-tabbing relates to two columns/ series together...

pd.crosstab(adata.obs["donor"], adata.obs["leiden"])

In [ ]:
########   generating it as frequency...
# axis : t specifies the axis along which the means are computed
# https://stackoverflow.com/questions/22149584/what-does-axis-in-pandas-mean
#   Usually axis=0 is said to be "column-wise" (and axis=1 "row-wise")

pd.crosstab(adata.obs["donor"], adata.obs["leiden"]).apply(
    lambda r: r / r.sum() * 100, axis=1
)

In [ ]:
#### Creating a stacked bar
pd.crosstab(adata.obs["sort_type"], adata.obs["leiden"]).apply(
    lambda r: r / r.sum() * 100, axis=1
).plot.bar(stacked=True)
plt.legend(bbox_to_anchor=(1, 0.5), loc="center left", frameon=False)

 - `bbox_to_anchor=(1, 0.5)`: This parameter specifies the position of the legend in relation to the axes. In this case, (1, 0.5) means the legend will be placed at the right side of the plot, and vertically centered (at 50% height of the plot).
 - `loc='center left'`: This parameter determines where the legend should be anchored within the bounding box specified by bbox_to_anchor. In this case, the legend will be positioned along the center-left edge of the bounding box.
 - `frameon=False`: This parameter specifies whether to draw a frame around the legend. By setting it to False, you're indicating that the legend should not have a visible frame.

In [ ]:
### Quickly plot Violin plots for QC...
sc.pl.violin(
    adata,
    ["total_counts", "n_genes_by_counts", "pct_counts_mt"],
    size=0.1,
    groupby="leiden",
    multi_panel=True,
    rotation=90,
)

# cluster 12 - a lot of mito genes, cluster 15 is basically non-existent!

In [ ]:
## check CD4/CD8 score over cluster.. - higher score = more CD4.. - cluster 2,3,4, 6,7,8, 11,13,14
sc.pl.matrixplot(adata, "CD4/CD8_geneset", groupby="leiden")

✋ This is in line with UMAP structure governed mainly by activation etc... Whereas lineage of CD4/CD8 of sorted identities are not distinct in the UMAP. CD4-like cells tend to sit on the upper half!

In [ ]:
sc.pl.violin(
    adata,
    ["CD4", "CD8A", "CD8B"],
    size=0.1,
    groupby="leiden",
    multi_panel=True,
    rotation=90,
)

# 6. Refining clusters based on marker gene expression
# (Manual way #1)

Using the Leiden clusters, rank the highly expressed genes for each cluster using a dot plot.
Try adding a new key to refine the number of clusters. Label it as "leiden_A"

In [ ]:
sc.tl.leiden(adata, key_added="leiden_A")
sc.pl.umap(
    adata, color=["leiden", "leiden_A"], legend_fontoutline=2, legend_loc="on data"
)

In [ ]:
sc.tl.rank_genes_groups(adata, groupby="leiden_A", method="wilcoxon")

In [ ]:
sc.pl.rank_genes_groups_dotplot(
    adata, n_genes=5, min_logfoldchange=0.5, standard_scale="var", color_map="viridis"
)

Now label each cluster with an annotation based on the most highly expressed genes while genes that are housekeeping or non-specific can be assigned to "Ambiguous" if they are not genes of interest.
Also consider the fraction of cells and their mean gene expression in tandem

You can also look at the genes in a tabular format for easier viewing. Still, use it in tandem with the dotplot above.

In [ ]:
pd.DataFrame(adata.uns["rank_genes_groups"]["names"]).head(10)

#### Now name the clusters based on the listed genes together with references to literature.

Notice that most of the clusters have lots of non-specific annd/or housekeeping genes that makes it hard to name the clusters.

So you can also just manually inspect and plot a few genes using a cell marker dictionary that we will then used to label the above clusters.

**I am leaving the next few lines of code partlially blank so that you can decide what kind genes to plot and how to annotate your celltypes. If you just run as it, you will encounter errors.**

In [ ]:
cellmarker_dict = {
    "T cell": ["CD3D", "TRAC"],
    "NK": ["NCAM1", "NCR1"],
    "CD4 vs CD8": ["CD4", "CD8B"],
    "Memory": ["SELL", "CCR7", "CD44"],
    "Th1": [],
    "Cytokines": [],
    "Chemokines": [],
    "Exhausted": [],
    "Other": [],
}

In [ ]:
sc.pl.dotplot(
    adata, cellmarker_dict, groupby="leiden_A", standard_scale="var", color_map="Blues"
)

### To map the identified clusters to a cell type annotation label

In [ ]:
celltypesannot = {
    "0": "",
    "1": "",
    "2": "",
    "3": "",
    "4": "",
    "5": "",
    "6": "",
    "7": "",
    "8": "",
    "9": "",
}
# add a new '.obs'column called 'cell type' by mapping clusters to annotation using pandas 'map'function
adata.obs["cell type"] = adata.obs["leiden_A"].map(celltypesannot).astype("category")

In [ ]:
sc.pl.umap(
    adata,
    color="cell type",
    legend_loc="on data",
    frameon=False,
    legend_fontsize=7,
    legend_fontoutline=2,
)

In [ ]:
adata2 = adata[
    adata.obs["cell type"].isin(
        [
            "",
            "",
            "",
        ]
    )
].copy()
sc.pl.umap(adata2, color="cell type")

In [ ]:
adata2.write("celltypes_annot_01.h5ad")

In [ ]:
adata2 = sc.read_h5ad("celltypes_annot_01.h5ad")  # run this if reimporting
adata2

So the above steps would be how you run a manual annotation and you can refine the clusters further using your newly labeled object "adata2"

# 7. Other ways of refining cluster annotations:

## Alternative method of cluster annotations: Automated Celltypist annotation

### First, write out the raw data for input for celltypist

In [ ]:
adata.raw.to_adata().write_h5ad(
    "adata_processed_v2_BBKNN_MS_raw-for_celltypist_300823.h5ad", compression="gzip"
)

#### Quick note... Kelvin also did a comparison on the pre-batch-correction data, but since we are happy with batch correction.. just continue on.

### Make a directory for the output after running celltypist on our data

In [ ]:
!mkdir celltypist_bbknn

#### Run celltypist on our processed-BBKNN-batch corrected data and allow it to predict labels on each single cell with all the specifications needed.

#### The labels will be given a variable name of "majority_voting"

In [ ]:
!celltypist --indata adata_processed_v2_BBKNN_MS_raw-for_celltypist_300823.h5ad --model Immune_All_Low.pkl --outdir celltypist_bbknn --majority-voting

#### Import and label the object for predicted_labels.csv as "celltypistdf1" and set an index_col=0.

index_col: This is to allow you to set which columns to be used as the index of the dataframe. The default value is None, and pandas will add a new column start from 0 to specify the index column. It can be set as a column name or column index, which will be used as the index column.

#### Then run the new object

majority_voting: predicted labels after majority voting in local subclusters

In [ ]:
celltypistdf1 = pd.read_csv("celltypist_bbknn/predicted_labels.csv", index_col=0)
celltypistdf1

#### Check the columns of adata:

In [ ]:
adata.obs

#### Include the majority_voting data from celltypistdf1 in the adata - Label it as "celltypist_majority_voting"

In [ ]:
adata.obs["celltypist_majority_voting"] = pd.Series(celltypistdf1["majority_voting"])
adata.obs.info()

#### Check the columns that you've added the new column in correctly:

In [ ]:
adata.obs

#### Data can also be imported this way without "pd.Series" at the start - not sure what difference is!:

adata.obs["celltypist_majority_voting"] = celltypistdf1["majority_voting"]

#### Visualise the data via umap

##### Hint: color it by your new celltypist labels

In [ ]:
sc.pl.umap(adata, color=["celltypist_majority_voting"], size=20)

### Plot distribution by sorting by cell type

In [ ]:
pd.crosstab(adata.obs["sort_type"], adata.obs["celltypist_majority_voting"]).apply(
    lambda r: r / r.sum() * 100, axis=1
).plot.bar(stacked=True)
plt.legend(bbox_to_anchor=(1, 0.5), loc="center left", frameon=False)

### Plot a crosstab chart by sorting into CD4, CD8 and DP using the package matplotlib.pyplot:

In [ ]:
import matplotlib.pyplot as plt

pd.crosstab(
    adata.obs["celltypist_majority_voting"],
    adata.obs["sort_type"],
).apply(
    lambda r: r / r.sum() * 100, axis=1
).plot.bar(stacked=True)
plt.legend(bbox_to_anchor=(1, 0.5), loc="center left", frameon=False)

### Plot a UMAP and colour it by leiden and place the legend 'on data'

In [ ]:
sc.pl.umap(adata, color="leiden", legend_loc="on data", legend_fontoutline=2, size=10)

In [ ]:
pd.crosstab(adata.obs["leiden"], adata.obs["celltypist_majority_voting"]).apply(
    lambda r: r / r.sum() * 100, axis=1
).plot.bar(stacked=True)
plt.legend(bbox_to_anchor=(1, 0.5), loc="center left", frameon=False)

At the end of all these, a theme has emerged:
CD4 sorted cells are mostly "naive"-like
CD8 sorted cells are mostly "cytotoxic"
DP are more related to CD4s, but have some differentianted Th cells

#### But if this is the case, none of the current annotation will have DP cells as a whole - the question is are DP cells more related to CD4/CD8 cells. So... we will group all DP cells and annotate them as DP cells without further clustering etc.

#### So next, create a new column in adata called  "celltypist_annotation":

In [ ]:
adata.obs["celltypist_annotation"] = adata.obs["celltypist_majority_voting"]
adata.obs.info()

In [ ]:
# convert from category to object so I can play around...
adata.obs["celltypist_annotation"] = adata.obs["celltypist_annotation"].astype("object")
adata.obs.info()

In [ ]:
### This function works only on object but NOT category!
for i in adata.obs.index:
    if adata.obs.at[i, "sort_type"] == "DP":
        adata.obs.at[i, "celltypist_annotation"] = "DP"
adata.obs.info()

In [ ]:
adata.obs

### Plot a umap and color it by the celltypist annotations.

In [ ]:
sc.pl.umap(adata, color=["celltypist_annotation"], size=20, ncols=2)

## Stop here.... and save and do marker genes etc....

In [ ]:
adata.write_h5ad(
    "adata_processed_v2_BBKNN_celltypist_MS_300823.h5ad", compression="gzip"
)

In [ ]:
adata = sc.read_h5ad("adata_processed_v2_BBKNN_celltypist_MS_300823.h5ad")
#### NOTE: This step is only necessary if you are running from any of the SAVED H5ad object, because adata.uns['log1p'] is an empty dictionary! So will trigger error when running any DGEA..
# pop is used to remove and return an element from a collection (if assigned to something), here it is to simply remove log1p
try:
    adata.uns.pop("log1p")
except:
    pass
adata

## Finding Marker Genes Manually

In [ ]:
sc.tl.rank_genes_groups(adata, groupby="leiden", method="wilcoxon")

Plot the above out:

In [ ]:
sc.pl.rank_genes_groups(adata, n_genes=25, sharey=False)  # sharey means each panel

Plot it out as dotplot:

In [ ]:
sc.pl.rank_genes_groups_dotplot(
    adata, n_genes=5, standard_scale="var", color_map="viridis", min_logfoldchange=1
)

Let's try to plot my marker genes of "CD4", "CD8B", "FOXP3", "CTLA4", "SELL", "CCR7", "MKI67", "NKG7", "GATA3", "RORC", "CXCR5", "CD69", "GZMK", "BCL6", "TRAV1-2" - label this set of genes as "my_genes", and plot it as a dotplot

In [ ]:
my_genes = [
    "CD4",
    "CD8B",
    "FOXP3",
    "CTLA4",
    "SELL",
    "CCR7",
    "MKI67",
    "NKG7",
    "GATA3",
    "RORC",
    "CXCR5",
    "CD69",
    "GZMK",
    "BCL6",
    "TRAV1-2",
]
sc.pl.dotplot(adata, var_names=my_genes, groupby="leiden", color_map="viridis")

Plot Violin plots for QC using the stacked_violin function:

In [ ]:
sc.pl.stacked_violin(
    adata,
    my_genes,
    groupby="leiden",
    row_palette="viridis",
    colorbar_title="median expression",
)

Plot a stacked_violin chart again but groupby as 'celltypist_annotation'

In [ ]:
sc.pl.stacked_violin(
    adata,
    my_genes,
    groupby="celltypist_annotation",
    row_palette="viridis",
    colorbar_title="median expression",
)

Show the top 10 ranked genes for each cluster as a dataframe using the `rank_genes_groups` function

In [ ]:
pd.DataFrame(adata.uns["rank_genes_groups"]["names"]).head(10)

Plot the umap and color by 'leiden' and set a size of 20:

In [ ]:
sc.pl.umap(adata, color="leiden", size=20)

Redo the above steps with celltypist_annotation:

In [ ]:
sc.tl.rank_genes_groups(adata, groupby="celltypist_annotation", method="wilcoxon")

Plot it out as dotplot with rank_genes_groups

In [ ]:
sc.pl.rank_genes_groups_dotplot(
    adata, n_genes=10, standard_scale="var", color_map="viridis", min_logfoldchange=1
)

Plot violin plots for QC with 'my_genes':

In [ ]:
my_genes = [
    "CD4",
    "CD8B",
    "FOXP3",
    "IL2RA",
    "CTLA4",
    "GZMK",
    "NKG7",
    "CCL5",
    "SELL",
    "CCR7",
    "CD44",
    "CD27",
    "TRAV1-2",
    "KLRG1",
    "CX3CR1",
    "CD69",
    "ITGA1",
    "CXCR6",
]
sc.pl.stacked_violin(
    adata,
    my_genes,
    groupby="celltypist_annotation",
    row_palette="viridis",
    colorbar_title="median expression",
)


Plot umap with clusters coloured by celltypist_annotation

In [ ]:
sc.pl.umap(adata, color="celltypist_annotation", size=10)

Show the top 10 ranked genes for each cluster as a dataframe using 'rank_genes_groups'

In [ ]:
pd.DataFrame(adata.uns["rank_genes_groups"]["names"]).head(10)

Do you want to try and give each cluster your own annotation and see if it agrees with the celltypist ones?

This marks the end of the tutorial. Good job!